In [5]:
import os
import librosa
import argparse
import numpy as np
import pandas as pd
from pathlib import Path
from pesq import pesq
from pystoi import stoi
import mir_eval
from scipy.io import wavfile
from joblib import Parallel, delayed
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def getSeparationMetrics(audio1, audio2, audio1_gt, audio2_gt):
    duration = int(2.55 * 16000)
    reference_sources = np.stack((audio1_gt[:duration], audio2_gt[:duration]))
    estimated_sources = np.stack((audio1[:duration], audio2[:duration]))
    sdr, sir, sar, _ = mir_eval.separation.bss_eval_sources(reference_sources, estimated_sources, False)
    return np.mean(sdr), np.mean(sir), np.mean(sar)

def process_folder(results_dir, audio_sampling_rate, mic_index=0):
    wav_files = [f for f in os.listdir(results_dir) if f.endswith('.wav') and f'mic{mic_index}' in f]
    separated_files = [f for f in wav_files if 'separated' in f]
    ground_truth_files = [f for f in wav_files if 'separated' not in f and 'mixed' not in f]

    if len(separated_files) != 2 or len(ground_truth_files) != 2:
        logging.warning(f"Skipping folder {results_dir} due to mismatched file counts.")
        return None

    separated_files.sort()
    ground_truth_files.sort()

    audio1_gt_path = Path(results_dir) / ground_truth_files[0]
    audio2_gt_path = Path(results_dir) / ground_truth_files[1]
    audio1_path = Path(results_dir) / separated_files[0]
    audio2_path = Path(results_dir) / separated_files[1]

    try:
        _, audio1 = wavfile.read(audio1_path)
        _, audio2 = wavfile.read(audio2_path)
        _, audio1_gt = wavfile.read(audio1_gt_path)
        _, audio2_gt = wavfile.read(audio2_gt_path)

        sdr, sir, sar = getSeparationMetrics(audio1, audio2, audio1_gt, audio2_gt)

        stoi_score1 = stoi(audio1_gt, audio1, audio_sampling_rate, extended=False)
        stoi_score2 = stoi(audio2_gt, audio2, audio_sampling_rate, extended=False)
        stoi_score = (stoi_score1 + stoi_score2) / 2

        pesq_score1 = pesq(audio_sampling_rate, audio1, audio1_gt, 'wb')
        pesq_score2 = pesq(audio_sampling_rate, audio2, audio2_gt, 'wb')
        pesq_score = (pesq_score1 + pesq_score2) / 2

        return {
            'Folder': os.path.basename(results_dir),
            'SDR': sdr,
            'SIR': sir,
            'SAR': sar,
            'PESQ': pesq_score,
            'STOI': stoi_score
        }
    except MemoryError as me:
        logging.error(f"Memory error processing folder {results_dir}: {me}")
        return None
    except Exception as e:
        logging.error(f"Error processing folder {results_dir}: {e}")
        return None

def process_folders(folders, audio_sampling_rate, mic_index, n_jobs=5):
    results = Parallel(n_jobs=n_jobs, backend='loky')(
        delayed(process_folder)(folder, audio_sampling_rate, mic_index) for folder in folders
    )
    processed_results = []
    for result, folder in zip(results, folders):
        if result is not None:
            processed_results.append(result)
        else:
            logging.warning(f"Retrying folder {folder} sequentially.")
            result = process_folder(folder, audio_sampling_rate, mic_index)
            if result:
                processed_results.append(result)
    return processed_results

def main():
    test_dir = "E:/AV-speech-separation/data/VoxCeleb2/results/simulated/no_phase"
    audio_sampling_rate = 16000
    mic_index = 0
    i = 251
    j = 255

    results_df = pd.DataFrame(columns=['Folder', 'SDR', 'SIR', 'SAR', 'PESQ', 'STOI'])
    test_dirs = [os.path.join(test_dir, d) for d in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, d))][i:j]

    n_jobs = 2
    results = process_folders(test_dirs, audio_sampling_rate, mic_index, n_jobs)

    results_df = pd.DataFrame(results)
    averages = results_df.mean(numeric_only=True)
    averages['Folder'] = 'Average'
    results_df = pd.concat([results_df, pd.DataFrame([averages])], ignore_index=True)

    excel_path = os.path.join(test_dir, f'evaluation_results_microphone_{mic_index}_{i}_{j}_no_phase.xlsx')
    results_df.to_excel(excel_path, index=False)
    logging.info(f"Results saved to {excel_path}")

if __name__ == '__main__':
    main()


2024-06-09 21:22:50,998 - INFO - Results saved to E:/AV-speech-separation/data/VoxCeleb2/results/simulated/no_phase\evaluation_results_microphone_0_251_255_no_phase.xlsx
